In [151]:
import decode.ml_logic.emotion_detection.emotion_baseline as ed

In [152]:
from tensorflow.keras.utils import image_dataset_from_directory
import numpy as np
import pandas as pd
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import models, Sequential, layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf


In [153]:
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input

In [189]:
train_ds = ed.load_data_local(ed.DIR_TRAIN, color_mode="rgb")

Found 28709 files belonging to 7 classes.


In [190]:
test_ds = ed.load_data_local(ed.DIR_TEST, color_mode="rgb")

Found 7178 files belonging to 7 classes.


In [191]:
base_model = InceptionResNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(95, 95, 3)
)
base_model.trainable = False

model = Sequential(
    [
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.2),
        layers.Dense(1, activation="softmax"),
    ]
)

In [192]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Functi  (None, 1, 1, 1536)       54336736  
 onal)                                                           
                                                                 
 global_average_pooling2d_13  (None, 1536)             0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dropout_13 (Dropout)        (None, 1536)              0         
                                                                 
 dense_13 (Dense)            (None, 7)                 10759     
                                                                 
Total params: 54,347,495
Trainable params: 10,759
Non-trainable params: 54,336,736
_________________________________________________________________


In [193]:
es = EarlyStopping(patience=5, restore_best_weights=True)

In [194]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=10000,
    decay_rate=0.9)

In [195]:
adam = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

In [205]:
#use this for ohe-labels
#model.compile(optimizer=adam, loss="CategoricalCrossentropy", metrics=["CategoricalAccuracy"])
model.compile(optimizer=adam, loss="SparseCategoricalCrossentropy", metrics=["SparseCategoricalAccuracy"])

In [209]:
history = model.fit(X_train, y_train, epochs=5, batch_size=32, callbacks=[es], validation_split=0.3)
#history = model.fit(train_ds, epochs=5, batch_size=32, callbacks=[es], validation_data=test_ds)

Epoch 1/5
35/35 [==============================] - 78s 2s/step - loss: 212.1270 - sparse_categorical_accuracy: 0.1929 - val_loss: 212.7259 - val_sparse_categorical_accuracy: 0.1349
Epoch 2/5
35/35 [==============================] - 89s 3s/step - loss: 241.3507 - sparse_categorical_accuracy: 0.1911 - val_loss: 184.7261 - val_sparse_categorical_accuracy: 0.1803
Epoch 3/5
35/35 [==============================] - 83s 2s/step - loss: 265.4041 - sparse_categorical_accuracy: 0.1786 - val_loss: 347.6176 - val_sparse_categorical_accuracy: 0.1531
Epoch 4/5
35/35 [==============================] - 89s 3s/step - loss: 407.9287 - sparse_categorical_accuracy: 0.1518 - val_loss: 616.2366 - val_sparse_categorical_accuracy: 0.1739
Epoch 5/5
35/35 [==============================] - 82s 2s/step - loss: 395.8302 - sparse_categorical_accuracy: 0.1848 - val_loss: 273.2176 - val_sparse_categorical_accuracy: 0.1879


In [221]:
model.predict(np.expand_dims(X_train[2],0))

1/1 [==============================] - 0s 53ms/step


array([[2.2151478e-06, 0.0000000e+00, 0.0000000e+00, 8.5687023e-01,
        1.4312749e-01, 0.0000000e+00, 0.0000000e+00]], dtype=float32)

In [231]:
int(y_train[2])

5

In [235]:
model.evaluate(X_train, y_train)

50/50 [==============================] - 14s 279ms/step - loss: 257.6644 - sparse_categorical_accuracy: 0.1894


[257.66436767578125, 0.18937499821186066]

In [212]:
model.predict(X_test[0])

ValueError: in user code:

    File "/Users/retter/.pyenv/versions/3.10.6/envs/decode_me/lib/python3.10/site-packages/keras/engine/training.py", line 2169, in predict_function  *
        return step_function(self, iterator)
    File "/Users/retter/.pyenv/versions/3.10.6/envs/decode_me/lib/python3.10/site-packages/keras/engine/training.py", line 2155, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/retter/.pyenv/versions/3.10.6/envs/decode_me/lib/python3.10/site-packages/keras/engine/training.py", line 2143, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/retter/.pyenv/versions/3.10.6/envs/decode_me/lib/python3.10/site-packages/keras/engine/training.py", line 2111, in predict_step
        return self(x, training=False)
    File "/Users/retter/.pyenv/versions/3.10.6/envs/decode_me/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(f

In [208]:
X_train, y_train = ed.get_first_batches(train_ds, 50);

In [165]:
X_test, y_test = ed.get_first_batches(test_ds, 20);

2023-05-25 19:47:53.991514: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [7178]
	 [[{{node Placeholder/_4}}]]
2023-05-25 19:47:53.991967: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [7178]
	 [[{{node Placeholder/_4}}]]


In [174]:
y_cat_train = to_categorical(y_train, num_classes=7)
y_cat_test = to_categorical(y_test, num_classes=7)

In [175]:
X_train_preproc = preprocess_input(X_train)

In [176]:
history = model.fit(X_train_preproc, y_cat_train, epochs=5, batch_size=32, callbacks=[es], validation_split=0.3, wei)
#history = model.fit(train_ds, epochs=5, batch_size=32, callbacks=[es])

Epoch 1/5


TypeError: 'NoneType' object is not callable

************************************